# Building MDPs
A Markov Decision Process is an extended Markov Chain which

* states
* actions
* transitions

Where

* actions are essentially 'decisions' which might be decided by *schedulers*
* each state has a number of actions
* actions have probabilistic transitions to states

A famous example of an MDP is the Monty Hall problem. We will show how to construct it using the pgc API.

In [11]:
from stormvogel import pgc
from stormvogel.model import EmptyAction, ModelType
from stormvogel.show import show
from stormvogel.layout import Layout, DEFAULT

init = pgc.State(x="")

empty_action = pgc.Action(["emp"])

def available_actions(s: pgc.State):
    if "carchosen" in s.x:
        return [pgc.Action(["choosedoor"]) for i in range(1,4)]
    elif "chosen" in s.x:
        return [pgc.Action(["stay"]), pgc.Action(["switch"])]
    return [empty_action]
    

def delta(s: pgc.State, a: pgc.Action):
    # empty actions
    if s == init:
        return [(1/3, pgc.State(x=["carchosen"], car=i)) for i in range(1,4)]
    elif "choosedoor" in a.labels:
        return [(1/3, pgc.State(x=["chose", str(i)], car=s.car, chosen=i)) for i in [1,2,3]]
    elif "chose" in s.x:
        possible_goats = set([1,2,3]) - {s.car} - {s.chosen}
        return [(1/len(possible_goats), 
                pgc.State(x=["chosen", str(s.chosen), f"goat,{i}"], 
                car=s.car, goat=i, chosen=s.chosen)) 
                for i in possible_goats]
    elif "stay" in a.labels:
        return [(1, pgc.State(x=["won"], pred=s))] if s.chosen == s.car else [(1, pgc.State(x=["lost"], pred=s))]
    elif "switch" in a.labels:
        return [(1, pgc.State(x=["won"], pred=s))] if s.chosen != s.car else [(1, pgc.State(x=["lost"], pred=s))]
    else:
        return [(1, s)]
        
# Labels is a function that tells the pgc API to use x as the label.
# It is written as a lambda here, but it means exactly the same as
# def labels(s):
#    return s.x
labels = lambda s: s.x

pgc_die = pgc.build_pgc(
    delta=delta,
    initial_state_pgc=init,
    available_actions=available_actions,
    labels=labels,
    modeltype=ModelType.MDP
)

vis2 = show(pgc_die)
# TODO finish this example once we know how to specify emtpy actions.